In [71]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_classif

In [39]:
df = pd.read_csv("heart.csv")

In [67]:
# Crear un diccionario de modelos
models = {
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=110, max_depth=3, warm_start=True ),
    "Random Forest": RandomForestClassifier(criterion='entropy', n_estimators=60, max_depth=3, bootstrap=True),
    "Bagging": BaggingClassifier(
        n_estimators= 100,max_samples=0.5, warm_start=True),
    "Voting": VotingClassifier(estimators=[
        ('knn', KNeighborsClassifier(weights='uniform')),
        ('logreg', LogisticRegression()),
        ('svc', SVC(probability=True)),
        ('nb', GaussianNB())
    ]),
    "Stacking": StackingClassifier(estimators=[
        ('knn', KNeighborsClassifier()),
        ('logreg', LogisticRegression()),
        ('svc', SVC(probability=True)),
        ('nb', GaussianNB())
    ], final_estimator=LogisticRegression()),
    "AdaBoost": AdaBoostClassifier(algorithm='SAMME')
}

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Listas para almacenar las métricas
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Iterar sobre los modelos
for name, model in models.items():
    # Realizar validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
    
    # Calcular las métricas
    accuracy = cv_scores.mean()
    precision = cross_val_score(model, X_train, y_train, cv=10, scoring='precision').mean()
    recall = cross_val_score(model, X_train, y_train, cv=10, scoring='recall').mean()
    f1 = cross_val_score(model, X_train, y_train, cv=10, scoring='f1').mean()
    
    # Ajustar el modelo al conjunto de entrenamiento completo
    model.fit(X_train, y_train)
    
    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)
    
    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Agregar las métricas a las listas
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    # Imprimir resultados
    print(f"Modelo: {name}")
    print(f"Accuracy (CV): {accuracy:.4f}")
    print(f"Precisión (CV): {precision:.4f}")
    print(f"Recall (CV): {recall:.4f}")
    print(f"F1 Score (CV): {f1:.4f}")
    print(f"Matriz de Confusión:")
    print(conf_matrix)
    print("="*30)

# Imprimir promedios
print(f"Promedio de Accuracy: {sum(accuracy_scores)/len(accuracy_scores):.4f}")
print(f"Promedio de Precisión: {sum(precision_scores)/len(precision_scores):.4f}")
print(f"Promedio de Recall: {sum(recall_scores)/len(recall_scores):.4f}")
print(f"Promedio de F1 Score: {sum(f1_scores)/len(f1_scores):.4f}")

Modelo: Gradient Boosting
Accuracy (CV): 0.8013
Precisión (CV): 0.8375
Recall (CV): 0.8187
F1 Score (CV): 0.8213
Matriz de Confusión:
[[24  5]
 [ 7 25]]
Modelo: Random Forest
Accuracy (CV): 0.8178
Precisión (CV): 0.8257
Recall (CV): 0.8940
F1 Score (CV): 0.8387
Matriz de Confusión:
[[24  5]
 [ 4 28]]
Modelo: Bagging
Accuracy (CV): 0.8055
Precisión (CV): 0.8172
Recall (CV): 0.8269
F1 Score (CV): 0.8285
Matriz de Confusión:
[[24  5]
 [ 3 29]]
Modelo: Voting
Accuracy (CV): 0.7815
Precisión (CV): 0.8094
Recall (CV): 0.7978
F1 Score (CV): 0.7989
Matriz de Confusión:
[[26  3]
 [ 5 27]]
Modelo: Stacking
Accuracy (CV): 0.8018
Precisión (CV): 0.8080
Recall (CV): 0.8423
F1 Score (CV): 0.8222
Matriz de Confusión:
[[25  4]
 [ 4 28]]
Modelo: AdaBoost
Accuracy (CV): 0.8137
Precisión (CV): 0.8348
Recall (CV): 0.8258
F1 Score (CV): 0.8282
Matriz de Confusión:
[[26  3]
 [ 8 24]]
Promedio de Accuracy: 0.8036
Promedio de Precisión: 0.8221
Promedio de Recall: 0.8342
Promedio de F1 Score: 0.8230


En la siguiente tabla podemos observar como se desempeñaron los algoritmos. Estos datos es ya habiendo probado varios parametros distintos. 

Modelo            | Accuracy | Precisión | Recall  | F1 Score 
-------------------|----------|-----------|---------|----------
Gradient Boosting | 0.8095   | 0.8323    | 0.8187  | 0.8174   
Random Forest     | 0.8178   | 0.8257    | 0.8940  | 0.8387   
Bagging           | 0.8055   | 0.8172    | 0.8269  | 0.8285   
Voting            | 0.7815   | 0.8094    | 0.7978  | 0.7989   
Stacking          | 0.8018   | 0.8080    | 0.8423  | 0.8222   
AdaBoost          | 0.8137   | 0.8348    | 0.8258  | 0.8282   

Es notorio que los algoritmos sufren un poco, especialmente voting que tiene unas metricas muy pobres. Se probo realizar un PCA antes pero eso empeoro los resultados por lo que se opto por probar ganancia de información.

In [72]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calcular la ganancia de información
info_gains = mutual_info_classif(X_train, y_train)

# Definir un umbral para la selección de características
umbral = 0.1
caracteristicas_seleccionadas = X_train.columns[info_gains > umbral]
X_train_seleccionado = X_train[caracteristicas_seleccionadas]
X_test_seleccionado = X_test[caracteristicas_seleccionadas]

# Crear un diccionario de modelos
models = {
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=110, max_depth=3, warm_start=True),
    "Random Forest": RandomForestClassifier(criterion='entropy', n_estimators=60, max_depth=3, bootstrap=True),
    "Bagging": BaggingClassifier(n_estimators=100, max_samples=0.5, warm_start=True),
    "Voting": VotingClassifier(estimators=[
        ('knn', KNeighborsClassifier(weights='uniform')),
        ('logreg', LogisticRegression()),
        ('svc', SVC(probability=True)),
        ('nb', GaussianNB())
    ]),
    "Stacking": StackingClassifier(estimators=[
        ('knn', KNeighborsClassifier()),
        ('logreg', LogisticRegression()),
        ('svc', SVC(probability=True)),
        ('nb', GaussianNB())
    ], final_estimator=LogisticRegression()),
    "AdaBoost": AdaBoostClassifier(algorithm='SAMME')
}

# Listas para almacenar las métricas
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Iterar sobre los modelos
for name, model in models.items():
    # Entrenar el modelo con características seleccionadas
    model.fit(X_train_seleccionado, y_train)
    
    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test_seleccionado)
    
    # Calcular las métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Agregar las métricas a las listas
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    # Imprimir resultados
    print(f"Modelo: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Matriz de Confusión:")
    print(conf_matrix)
    print("="*30)

# Imprimir promedios
print(f"Promedio de Accuracy: {sum(accuracy_scores)/len(accuracy_scores):.4f}")
print(f"Promedio de Precisión: {sum(precision_scores)/len(precision_scores):.4f}")
print(f"Promedio de Recall: {sum(recall_scores)/len(recall_scores):.4f}")
print(f"Promedio de F1 Score: {sum(f1_scores)/len(f1_scores):.4f}")

Modelo: Gradient Boosting
Accuracy: 0.8033
Precisión: 0.7941
Recall: 0.8438
F1 Score: 0.8182
Matriz de Confusión:
[[22  7]
 [ 5 27]]
Modelo: Random Forest
Accuracy: 0.8525
Precisión: 0.8710
Recall: 0.8438
F1 Score: 0.8571
Matriz de Confusión:
[[25  4]
 [ 5 27]]
Modelo: Bagging
Accuracy: 0.8525
Precisión: 0.8485
Recall: 0.8750
F1 Score: 0.8615
Matriz de Confusión:
[[24  5]
 [ 4 28]]
Modelo: Voting
Accuracy: 0.8689
Precisión: 0.8750
Recall: 0.8750
F1 Score: 0.8750
Matriz de Confusión:
[[25  4]
 [ 4 28]]
Modelo: Stacking
Accuracy: 0.8852
Precisión: 0.8788
Recall: 0.9062
F1 Score: 0.8923
Matriz de Confusión:
[[25  4]
 [ 3 29]]
Modelo: AdaBoost
Accuracy: 0.8525
Precisión: 0.8966
Recall: 0.8125
F1 Score: 0.8525
Matriz de Confusión:
[[26  3]
 [ 6 26]]
Promedio de Accuracy: 0.8525
Promedio de Precisión: 0.8607
Promedio de Recall: 0.8594
Promedio de F1 Score: 0.8594


Podemos ver que las metricas aumentaron mejoraron bastante en especial stacking y voting que son los dos mejores modelos en cuanto a su balance. 

Modelo            | Accuracy | Precisión | Recall  | F1 Score 
-------------------|----------|-----------|---------|----------
Gradient Boosting | 0.8033   | 0.7941    | 0.8438  | 0.8182   
Random Forest     | 0.8525   | 0.8710    | 0.8438  | 0.8571   
Bagging           | 0.8525   | 0.8485    | 0.8750  | 0.8615   
Voting            | 0.8689   | 0.8750    | 0.8750  | 0.8750   
Stacking          | 0.8852   | 0.8788    | 0.9062  | 0.8923   
AdaBoost          | 0.8525   | 0.8966    | 0.8125  | 0.8525   

Conluyendo, podemos decir que los dos mejores modelos tienen parametros competitivos superando a los modelos de la practica pasada. Esto no quiere decir que sean mejores  dado que anteriormente no se utilizo information gain por lo que no seria una comparativa justa. 
Aunque los modelos son relativamente buenos, no son capaces de superar el umbral de las metricas en mas de 90 por lo que podria decirse que les falta mas refinamiento para este caso especifico. 